<a href="https://colab.research.google.com/github/DCEG-workshops/statgen_workshop_tutorial/blob/main/src/04_Heritability_PRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up Google Drive

***Important***: We want to mount the *google drive* for the data neeed for this workshop. Note that this folder is different from previous lectures. Please open this [link](https://drive.google.com/drive/folders/13RlwRIlLmXFeWxB1elz6srb5Eip5HyAU?usp=sharing) with your Google drive and find the "statgen_workshop_04Heritability_PRS" folder under "Share with me". Then add a shortcut to the folder under "My Drive".

Mount Google drive

In [9]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Set up path variables

In [11]:
import os
input_dir="drive/MyDrive/statgen_workshop_04Heritability_PRS/"
analysis_dir=os.getcwd() + "/04_analysis/"
os.environ['input_dir']=input_dir
os.environ['analysis_dir']=analysis_dir

Take a look at data in input_dir

In [15]:
%%bash
ls ${input_dir}/data/

1000G_Phase3_baselineLD_ldscores
1000G_Phase3_frq
1000G_Phase3_weights_hm3_no_MHC
1kg_eur_22
chr22.bed
chr22.bim
chr22.fam
EUR_sum_data
eur_w_ld_chr
lua_bc
overall_bc
prs_genotype
tn_bc
y_out


load R magic, so that we can run R here and share variables

In [16]:
%load_ext rpy2.ipython

Install conda in colab

In [4]:
import os

conda_path = "/usr/local/bin/conda"

if os.path.exists(conda_path):
    print(f"{conda_path} exists.")
else:
    print(f"{conda_path} does not exist, installing")
    !pip install -q condacolab
    import condacolab
    condacolab.install()

/usr/local/bin/conda does not exist, installing
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:08
🔁 Restarting kernel...


In [17]:
!conda --version

conda 23.1.0


Let's install gcta conda environment, this takes about 2 minutes

In [18]:
%%bash
conda install -c bioconda gcta=1.93.2beta

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - gcta=1.93.2beta


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boltons-23.0.0             |     pyhd8ed1ab_0         296 KB  conda-forge
    ca-certificates-2023.7.22  |       hbcca054_0         146 KB  conda-forge
    certifi-2023.7.22          |     pyhd8ed1ab_0         150 KB  conda-forge
    conda-23.3.1               |  py310hff52083_0         941 KB  conda-forge
    gcta-1.93.2beta            |       h9ee0642_1         8.7 MB  bioconda
    jsonpatch-1.33             |     pyhd8ed1ab_0          17 KB  conda-forge
    jsonpointer-2.4            |  py310hff52083_3          16 KB  conda-forge
    openssl-3.1.4              |       hd590300_0         2.5 MB  co

In [19]:
%%bash
gcta64

*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version 1.93.2 beta Linux
* (C) 2010-present, Jian Yang, The University of Queensland
* Please report bugs to Jian Yang <jian.yang.qt@gmail.com>
*******************************************************************
Analysis started at 23:48:37 UTC on Tue Oct 24 2023.
Hostname: d22aafada359

Error: no analysis has been launched by the option(s)
Please see online documentation at http://cnsgenomics.com/software/gcta


CalledProcessError: ignored

Running GCTA to get the heritability
Data were generated using 1000 Genomes European Data CHR 22 data
Heritability was set up as 0.2
Causal SNPs proportion: 5%

Step 1, Compute the Genetic relationship matrix (GRM)

In [21]:
%%R -i input_dir -i analysis_dir

system(paste0("gcta64 ",
              "--bfile ",input_dir,"data/chr22 ",
              "--make-grm ",
              "--out ",analysis_dir,"result/chr22"))

Step 2, Compute the Heritability

In [22]:
%%R
system(paste0("gcta64 ",
              "--reml ",
              "--grm ",analysis_dir,"result/chr22 ",
              "--pheno ",analysis_dir,"result/phenotype.phen ",
              "--grm-cutoff 0.05 ",
              "--out ",analysis_dir,"result/gcta_herit"))

# Use LDSC to estimate heritability
Data were obtained from the GWAS summary statistics of breast cancer
Three different traits were included: overall breast cancer risk, Luminal A, Triple negative
More background of the GWAS can be found in: https://www.nature.com/articles/s41588-020-0609-2

We will install the ldsc conda environment, this step will take 5 minutes+

In [ ]:
%%bash
git clone https://github.com/bulik/ldsc.git && cd ldsc && conda env create --file environment.yml

Activate the ldsc conda environment

In [57]:
%%bash
source activate ldsc

Add ldsc scripts to the PATH

In [70]:
import os
os.environ['PATH']=os.environ['PATH']+":$(pwd)/ldsc"

In [71]:
%%bash
echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:$(pwd)/ldsc


look at the data,
the data is based on breast cancer GWAS

- snpid; if the SNP has rs id, then snpid is rsid, otherwise snpid is chr:position
- CHR: chromosome
- bp: GRCh37 (hg19)
- A1: effect_allele
- A2: non_effect_allele
- Z: Z-statistics
- P: P-value
- info: imputation quality score
- MAF: minor allele frequency
- N: effective-sample size; N was calculated as 1/(var(beta)*2*f*(1-f))
- effective sample size is needed if one wants to calculate the logit-scale genetic variance

In [50]:
%%R
library(data.table)
bcac_overall <- fread(paste0(input_dir,"data/overall_bc"))
head(bcac_overall)

R[write to console]: data.table 1.14.8 using 1 threads (see ?getDTthreads).  Latest news: r-datatable.com

R[write to console]: |--------------------------------------------------|
|
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to c

         snpid CHR    bp A2 A1          Z          P     info       MAF
1:  chr1:11008   1 11008  G  C -1.9783658 0.04788746 0.579745 0.0974118
2:  chr1:11012   1 11012  G  C -1.9783608 0.04788802 0.579746 0.0974118
3: rs201725126   1 13116  G  T  0.7119430 0.47650010 0.388268 0.1751340
4: rs200579949   1 13118  G  A  0.7119523 0.47649430 0.388268 0.1751340
5:  chr1:13273   1 13273  C  G  0.8408968 0.40040577 0.356340 0.1435200
6:  chr1:14464   1 14464  T  A -0.6623217 0.50776508 0.346048 0.1634150
          N
1: 24184.71
2: 24184.71
3: 17920.88
4: 17920.88
5: 17208.96
6: 16821.18


In [72]:
%%R

system(paste0("munge_sumstats.py ",
              "--sumstats ",input_dir,"data/overall_bc ",
              "--out ",analysis_dir,"result/ldsc_herit_overall ",
              "--merge-alleles ",input_dir,"data/eur_w_ld_chr/w_hm3.snplist ",
              "--chunksize 500000 ",
              "--signed-sumstats Z,0 --info-min 0.3 --maf-min 0.01"))

munge_result = paste0(analysis_dir,"result/ldsc_herit_overall.sumstats.gz")

the frality scale heritablity is 0.4777

In [73]:
%%R
system(paste0("ldsc.py ",
       "--h2 ", munge_result," ",
       "--ref-ld-chr ", input_dir,"data/eur_w_ld_chr/ ",
       "--w-ld-chr ", input_dir,"data/eur_w_ld_chr/ ",
       "--out ", analysis_dir,"result/h2_overall "))

genetic correlation calculation for luminal A and triple negative breast cancer subtypes

munge the luminal A and triple negative summary statistics

In [6]:
%%bash
ls drive/MyDrive/statgen_workshop_04Heritability_PRS/data/

1000G_Phase3_baselineLD_ldscores
1000G_Phase3_frq
1000G_Phase3_weights_hm3_no_MHC
1kg_eur_22
chr22.bed
chr22.bim
chr22.fam
EUR_sum_data
eur_w_ld_chr
overall_bc
prs_genotype
y_out


In [79]:
%%R
lua <- fread(paste0(input_dir,"data/lua_bc"))
head(lua)
system(paste0("munge_sumstats.py ",
              "--sumstats ",input_dir,"data/lua_bc ",
              "--out ",analysis_dir,"result/ldsc_herit_lua ",
              "--merge-alleles ",input_dir,"data/eur_w_ld_chr/w_hm3.snplist ",
              "--chunksize 500000 ",
              "--signed-sumstats Z,0 --info-min 0.3 --maf-min 0.01"))

munge_result_lua = paste0(analysis_dir,"result/ldsc_herit_lua.sumstats.gz")

R[write to console]: Error in fread(paste0(input_dir, "data/lua_bc")) : 
  File 'drive/MyDrive/statgen_workshop_04Heritability_PRS/data/lua_bc' does not exist or is non-readable. getwd()=='/content'




Error in fread(paste0(input_dir, "data/lua_bc")) : 
  File 'drive/MyDrive/statgen_workshop_04Heritability_PRS/data/lua_bc' does not exist or is non-readable. getwd()=='/content'


munge the luminal A and triple negative summary statistics

In [ ]:
%%R
system(paste0("module load ldsc; ",
              "munge_sumstats.py ",
              "--sumstats ",cur_dir,"data/tn_bc ",
              "--out ",cur_dir,"result/ldsc_herit_tn ",
              "--merge-alleles ",cur_dir,"data/eur_w_ld_chr/w_hm3.snplist ",
              "--chunksize 500000 ",
              "--signed-sumstats Z,0 --info-min 0.3 --maf-min 0.01"))

munge_result_tn = paste0(cur_dir,"result/ldsc_herit_tn.sumstats.gz")

calculate genetic correlation

In [ ]:
%%R
system(paste0("module load ldsc; ",
              "ldsc.py ",
              "--rg ", munge_result_lua,",",munge_result_tn," ",
              "--ref-ld-chr ", cur_dir,"data/eur_w_ld_chr/ ",
              "--w-ld-chr ", cur_dir,"data/eur_w_ld_chr/ ",
              "--out ", cur_dir,"result/rg_lua_tn "))

#genetic correlation 0.4829 (s.e. 0.0512)

stratified LD-score regression using baseline annotation

In [ ]:
%%R
system(paste0("module load ldsc; ",
              "ldsc.py ",
              "--h2 ", munge_result," ",
              "--ref-ld-chr ", cur_dir,"data/1000G_Phase3_baselineLD_ldscores/baselineLD. ",
              "--w-ld-chr ", cur_dir,"data/1000G_Phase3_weights_hm3_no_MHC/weights.hm3_noMHC. ",
              "--overlap-annot  ",
              "--frqfile-chr ", cur_dir,"data/1000G_Phase3_frq/1000G.EUR.QC. ",
              "--out ", cur_dir,"result/h2_sldsc "))
enrichment_result = fread(paste0(analysis_dir,"result/h2_sldsc.results")
which.max(enrichment_result$Enrichment)
enrichment_result[13,]

ok, we are done with ldsc, and let's deactivate the ldsc environment

In [11]:
%%bash
eval "$(conda shell.bash hook)"
conda deactivate

Construct the PRS using clumping and thresholding
Data were generated using 1000 Genomes European population as the reference data
Summary statistics of CHR 22 were provided
PLINK 1.9 will be used for the clumping
PLINK 2.0 will be used for score calculation
R2 between PRS and Y will be calculated using R

Install plink 1.9 and 2.0 conda environments

In [12]:
%%bash
conda install -c bioconda plink
conda install -c bioconda plink2

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - plink


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    plink-1.90b6.21            |       h031d066_5         6.8 MB  bioconda
    ------------------------------------------------------------
                                           Total:         6.8 MB

The following NEW packages will be INSTALLED:

  plink              bioconda/linux-64::plink-1.90b6.21-h031d066_5 




Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - plink2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|------



==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.9.0




==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.9.0




clumping with windowsize 500kb, clumping r2 0.01, max p-value 1.0

In [13]:
%%R -i input_dir -i analysis_dir

library(glue)

sum_data_file = glue(input_dir, "data/EUR_sum_data")
ref_data = glue(input_dir, "data/1kg_eur_22/chr_22")
out_file = glue(analysis_dir, "result/LD_clump")

res = system(paste0("plink ",
"--bfile ",ref_data," ",
"--clump ",sum_data_file," ",
"--clump-p1 1 ",
"--clump-r2 0.1  ",
"--clump-kb 500 ",
"--out ",out_file))

load the clump result

In [15]:
%%R
library(data.table)
LD_clump = fread(paste0(out_file,".clumped"))[,3,drop=F]

R[write to console]: data.table 1.14.8 using 1 threads (see ?getDTthreads).  Latest news: r-datatable.com



load the summary statistics

In [16]:
%%R
EUR_sum = fread(sum_data_file)

match the LD clumping results with summary statistics

In [17]:
%%R
library(dplyr)
prs_prep = left_join(LD_clump,EUR_sum, by = "SNP")
head(prs_prep)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:data.table’:

    between, first, last


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




                        SNP CHR       BP A1 TEST  NMISS     BETA   STAT
1:   rs5761190:26189870:G:T  22 26189870  G  ADD 100000 -0.03959 -7.985
2: rs149657410:47598718:G:A  22 47598718  A  ADD 100000  0.13830  7.530
3:    rs738697:49026230:T:C  22 49026230  C  ADD 100000  0.03544  6.984
4:   rs7284488:40412551:G:C  22 40412551  G  ADD 100000 -0.02799 -6.158
5:   rs5752361:27027214:C:G  22 27027214  G  ADD 100000 -0.03996 -5.943
6:     rs93108:32463610:G:A  22 32463610  G  ADD 100000  0.02631  5.760
           P
1: 1.416e-15
2: 5.108e-14
3: 2.895e-12
4: 7.381e-10
5: 2.810e-09
6: 8.425e-09


SNPs are ranked from the smallest p-value to largest p-value

In [ ]:
%%R

pthres <- c(5E-08,5E-07,5E-06,5E-05,5E-04,5E-03,5E-02,5E-01,1.0)
for(k in 1:length(pthres)){
  prs_coeff = prs_prep %>%
    filter(P<=pthres[k]) %>%
    select(SNP, A1, BETA) %>%
    as.data.frame()

  write.table(prs_coeff,
              file = glue("{analysis_dir}/result/prs_coeff_{k}"),
              row.names = F,
              col.names = T,
              quote = F)

  geno_file = glue(input_dir, "data/prs_genotype/chr22_test")
  prs_coeff_file = paste0(analysis_dir, "/result/prs_coeff_",k)
 prs_out = paste0(analysis_dir, "/result/prs_",k)
  res <- system(paste0("plink2 ",
                       "--score ",prs_coeff_file," cols=+scoresums,-scoreavgs header no-mean-imputation  ",
                       "--bfile ",geno_file," --out ",prs_out))
}

- Evaluate the performance of
- We have 20,000 people for tuning and validation purpose
- ID:10,001-11,000 will be used for the tuning dataset: select best p-value thresholding cutoff
- ID:11,001-12,000 will be used for the validation dataset: report the final performance
- read the outcome

In [ ]:
%%R
y_out = fread(  glue(input_dir, "data/y_out"))
y_tun = y_out[1:10000,"y"]
y_vad = y_out[10001:20000,"y"]
#create a vector to same the performance
r2_vec_tun = rep(0,length(pthres))
for(k in 1:length(pthres)){

  prs = fread(glue("{analysis_dir}/result/prs_{k}.sscore"))

  prs_tun = prs$SCORE1_SUM[1:10000]
  model = lm(y_tun$y~prs_tun)
  r2_vec_tun[k] = summary(model)$r.squared
}

find best performance on the tuning dataset

In [ ]:
%%R
idx.max = which.max(r2_vec_tun)

evaluate it on the validation

In [ ]:
%%R
prs = fread(paste0("/data/BB_Bioinformatics/stat_gene_course/result/prs_",idx.max,".sscore"))
prs_vad = prs$SCORE1_SUM[10001:20000]
model = lm(y_vad$y~prs_vad)
r2 = summary(model)$r.squared
print(r2)